# Lecture 59: Activity recognition using 3D-CNN

## Saving frames of videos as pytorch tensors

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pickle

In [ ]:
# Load train-test list
with open('trainList_5class.pckl','rb') as f:
    trainList = pickle.load(f)
with open('testList_5class.pckl','rb') as f:
    testList = pickle.load(f)

In [ ]:
# Printing classes
classes = []
for item in trainList:
    c = item.split('_')[1]
    if c not in classes:
        classes.append(c)
print(classes)

In [ ]:
# PIL image to pytorch tensor transformation
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),            
        transforms.ToTensor()])

In [ ]:
framePath = 'frames/' # Directory containing the frames
# Pack frames of to single tensor per video
for item in trainList:
    cName = item.split('_')[1]
    srcPath = framePath+cName+'/'+item    
    fNames = os.listdir(srcPath)
    fTemplate = fNames[0].split('_')
    fCount = len(fNames)
    for fNum in range(fCount):
        fileName = fTemplate[0]+'_'+fTemplate[1]+'_'+fTemplate[2]+'_'+fTemplate[3]+'_'+str(fNum+1)+'.jpg'
        if os.path.exists(srcPath+'/'+fileName):
            # Load image
            img = Image.open(srcPath+'/'+fileName) 
            # Transform to tensor
            imgTensor = data_transforms(img).unsqueeze(1)                              
            # Stack tensors per video. trainTensor dimension: 3xframe_countx224x224
            if fNum == 0:
                trainTensor = imgTensor                
            else:
                trainTensor = torch.cat((trainTensor,imgTensor),1)
        else:
            print(fileName+ ' missing!')  
        
    # Directory structure: ucf101_vidTensors --> train-> class name -> tensors
    tensorSavePath = 'ucf101_vidTensors/train/'+cName
    if not os.path.exists(tensorSavePath):
        os.makedirs(tensorSavePath)
    torch.save(trainTensor,os.path.join(tensorSavePath,item+'.pt'))   
    

In [ ]:
# Pack frames of to single tensor per video
for item in testList:
    cName = item.split('_')[1]
    srcPath = framePath+cName+'/'+item    
    fNames = os.listdir(srcPath)
    fTemplate = fNames[0].split('_')
    fCount = len(fNames)
    for fNum in range(fCount):
        fileName = fTemplate[0]+'_'+fTemplate[1]+'_'+fTemplate[2]+'_'+fTemplate[3]+'_'+str(fNum+1)+'.jpg'
        if os.path.exists(srcPath+'/'+fileName):
            # Load image
            img = Image.open(srcPath+'/'+fileName)
             # Transform to tensor
            imgTensor = data_transforms(img).unsqueeze(1)             
            # Stack tensors per video. testTensor dimension: 3xframe_countx224x224
            if fNum == 0:
                testTensor = imgTensor                
            else:
                testTensor = torch.cat((testTensor,imgTensor),1)
        else:
            print(fileName+ ' missing!')       
        
    # Directory structure: ucf101_vidTensors --> train-> class name -> tensors
    tensorSavePath = 'ucf101_vidTensors/test/'+cName
    if not os.path.exists(tensorSavePath):
        os.makedirs(tensorSavePath)
    torch.save(testTensor,os.path.join(tensorSavePath,item+'.pt'))   
   